## GENERATING THE SKLEARN-COMPATIBLE CSV

Using pyMongo driver, the CSV file has been generated by creating a Pandas DataFrame with converted values for the classification

In [1]:
from pymongo import MongoClient
import pandas as pd
import json

client = MongoClient("mongodb://localhost:27017")
db = client.DataMining
collection = db.parsed_matches
matches = json.loads(json.dumps([match for match in collection.find({})], default=str))
client.close()

finalResult = []
itCount = 0

for match in matches:
  myDict = {}
  myDict["rank"] = match["rank"]
  
  if myDict["rank"] == "Fun Deck":
        continue
  

  # COUNTS
  myDict["crowns"] = match["crowns"]
  myDict["troop"] = match["troop"]
  myDict["building"] = match["building"]
  myDict["spell"] = match["spell"]
  myDict["common"] = match["common"]
  myDict["rare"] = match["rare"]
  myDict["epic"] = match["epic"]
  myDict["legendary"] = match["legendary"]
  myDict["elixir"] = match["elixir"]
  myDict["conf_synergy"] = match["conf_synergy"]
  myDict["lift_synergy"] = match["lift_synergy"]
  commonLevelSum = 0
  rareLevelSum = 0
  epicLevelSum = 0
  legendaryLevelSum = 0
    
  # END COUNTS

  # ELABORATIONS
  cardList = [match['card1'], match['card2'], match['card3'], match['card4'], match['card5'], match['card6'], match['card7'], match['card8']]
  levelSum = 0
  avgSpeed = 0
  avgSpellDuration = 0
  totalHP = 0
  totalDMG = 0
  avgAS = 0
  totalUnitsCount = 0
  abilityCount = 0
  avgRange = 0
  totalGroundT = 0
  totalAirT = 0
  totalBuildingT = 0
  totalBuffCount = 0
  totalProjectiles = 0
  for card in cardList:
    # TROOP
    if card["type"] == "Troop":
      levelSum += int(card["level"], 10)
      if card["rarity"] == "Common":
        commonLevelSum += int(card["level"], 10)
      if card["rarity"] == "Rare":
        rareLevelSum+=int(card["level"], 10)
      if card["rarity"] == "Epic":
        epicLevelSum+=int(card["level"], 10)
      if card["rarity"] == "Legendary":
        legendaryLevelSum+=int(card["level"],10)
      avgSpeed += card["speed"]
      totalHP += card["hitpoints"]
      totalDMG += card["damage"]
      avgAS += card["hps"]
      avgRange += card["range"]
      totalUnitsCount += card["count"]
      if card["ability"]:
        abilityCount += 1
      if card["attacks_ground"]:
        totalGroundT += 1
      if card["attacks_air"]:
        totalAirT += 1
      if card["target_only_buildings"]:
        totalBuildingT += 1
    # SPELL
    if card["type"] == "Spell":
      levelSum += int(card["level"], 10)
      if card["rarity"] == "Common":
        commonLevelSum += int(card["level"], 10)
      if card["rarity"] == "Rare":
        rareLevelSum+=int(card["level"], 10)
      if card["rarity"] == "Epic":
        epicLevelSum+=int(card["level"], 10)
      if card["rarity"] == "Legendary":
        legendaryLevelSum+=int(card["level"],10)
      if "duration" in card:
        avgSpellDuration += card["duration"]
      avgRange += card["radius"]
      if card["hits_ground"]:
        totalGroundT += 1
      if card["hits_air"]:
        totalAirT += 1
      if "appliesBuff" in card:
        if card["appliesBuff"]:
          totalBuffCount += 1
      if card["projectile"]:
        totalProjectiles += 1
      if "damage" in card:
        totalDMG += card["damage"]
      if "ignore_buildings" in card:
        if card["ignore_buildings"]:
          totalBuildingT += 1
    # BUILDING
    if card["type"] == "Building":
      levelSum += int(card["level"], 10)
      if card["rarity"] == "Common":
        commonLevelSum += int(card["level"], 10)
      if card["rarity"] == "Rare":
        rareLevelSum+=int(card["level"], 10)
      if card["rarity"] == "Epic":
        epicLevelSum+=int(card["level"], 10)
      if card["rarity"] == "Legendary":
        legendaryLevelSum+=int(card["level"],10)
      avgSpellDuration += card["duration"]
      avgRange += card["range"]
      totalHP += card["hitpoints"]
      if "damage" in card:
        totalDMG += card["damage"]
      if "hitspeed" in card:
        avgAS += card["hitspeed"]
      if card["attacks_ground"]:
        totalGroundT += 1
      if card["attacks_air"]:
        totalAirT += 1
    # END FOR
  # AVG CALCULATIONS
  if(match["troop"] > 0):
    avgSpeed = avgSpeed / match["troop"]
  if (match["troop"] + match["building"] > 0): 
    avgAS = avgAS / (match["troop"] + match["building"])
  if (match["spell"] + match["building"] > 0):
    avgSpellDuration = avgSpellDuration / (match["spell"] + match["building"])
  avgRange = avgRange / 8
  # END ELABORATIONS
  myDict["levelSum"] = levelSum
  myDict["avgSpeed"] = avgSpeed
  myDict["avgSpellDuration"] = avgSpellDuration
  myDict["totalHP"] = totalHP
  myDict["totalDMG"] = totalDMG
  myDict["avgAS"] = avgAS
  myDict["totalUnitsCount"] = totalUnitsCount
  myDict["abilityCount"] = abilityCount
  myDict["avgRange"] = avgRange
  myDict["totalGroundT"] = totalGroundT
  myDict["totalAirT"] = totalAirT
  myDict["totalBuildingT"] = totalBuildingT
  myDict["totalBuffCount"] = totalBuffCount
  myDict["totalProjectiles"] = totalProjectiles
  if (match["common"] > 0):
      myDict["commonLevel"] = commonLevelSum / match["common"]
  else:
      myDict["commonLevel"] = 0
        
  if match["rare"] > 0:
      myDict["rareLevel"] = rareLevelSum / match["rare"]
  else:
      myDict["rareLevel"] = 0

  if match["epic"] > 0:
      myDict["epicLevel"] = epicLevelSum / match["epic"]
  else:
      myDict["epicLevel"] = 0
        
  if match["legendary"] > 0:
      myDict["legendaryLevel"] = legendaryLevelSum / match["legendary"]
  else:
      myDict["legendaryLevel"] = 0

  finalResult.append(myDict)

matches_df = pd.DataFrame(data=finalResult)
#version1: ranks.csv
matches_df.to_csv("../csv_sources/leveled_for_rarity.csv", header=True)
matches_df

,rank,crowns,troop,building,spell,common,rare,epic,legendary,elixir,...,avgRange,totalGroundT,totalAirT,totalBuildingT,totalBuffCount,totalProjectiles,commonLevel,rareLevel,epicLevel,legendaryLevel
0,Challenger,2,5,0,3,1,1,4,2,3.375,...,2712.50,8,3,0,0,3,11.00,11.0,10.25,10.5
1,Challenger,1,5,1,2,1,2,3,2,4.250,...,2018.75,7,5,1,2,0,13.00,13.0,13.00,13.0
2,Master,1,7,1,0,3,3,2,0,4.875,...,1925.00,8,2,2,0,0,13.00,13.0,13.00,0.0
3,Challenger,1,7,0,1,4,2,1,1,3.750,...,4175.00,8,6,0,0,1,13.00,13.0,13.00,13.0
4,Challenger,3,5,0,3,1,1,5,1,4.500,...,3056.25,8,4,1,2,1,13.00,13.0,13.00,13.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,Champion,3,5,1,2,0,4,4,0,3.625,...,2800.00,8,4,1,1,1,0.00,13.0,13.00,0.0
10496,Champion,3,6,0,2,1,2,3,2,4.000,...,2581.25,8,4,1,2,0,13.00,13.0,13.00,13.0
10497,Champion,1,6,0,2,2,1,2,3,3.750,...,2275.00,8,4,1,1,1,13.00,13.0,13.00,13.0
10498,Champion,1,5,1,2,4,2,1,1,3.125,...,3868.75,8,3,1,0,2,16.25,13.0,0.00,13.0


## SPLITTER

In [2]:
from pymongo import MongoClient
import pandas as pd
import json

client = MongoClient("mongodb://localhost:27017")
db = client.DataMining
collection = db.leveled_matches
matches = json.loads(json.dumps([match for match in collection.find({})], default=str))
client.close()

finalResult = []
itCount = 0

for match in matches:
  myDict = {}
  myDict["rank"] = match["rank"]
  
  if myDict["rank"] == "Fun Deck":
        continue

  # COUNTS
  myDict["troop"] = match["troop"]
  myDict["building"] = match["building"]
  myDict["spell"] = match["spell"]
  myDict["common"] = match["common"]
  myDict["rare"] = match["rare"]
  myDict["epic"] = match["epic"]
  myDict["legendary"] = match["legendary"]
  myDict["elixir"] = match["elixir"]
  myDict["conf_synergy"] = match["conf_synergy"]
  myDict["lift_synergy"] = match["lift_synergy"]
  # END COUNTS

  # ELABORATIONS
  cardList = [match['card1'], match['card2'], match['card3'], match['card4'], match['card5'], match['card6'], match['card7'], match['card8']]
  levelSum = 0
  avgSpeed = 0
  avgSpellDuration = 0
  totalHP = 0
  totalDMG = 0
  avgAS = 0
  totalUnitsCount = 0
  abilityCount = 0
  avgRange = 0
  totalGroundT = 0
  totalAirT = 0
  totalBuildingT = 0
  totalBuffCount = 0
  totalProjectiles = 0
  for card in cardList:
    # TROOP
    if card["type"] == "Troop":
      levelSum += int(card["level"], 10)
      avgSpeed += card["speed"]
      totalHP += card["hitpoints"]
      totalDMG += card["damage"]
      avgAS += card["hps"]
      avgRange += card["range"]
      totalUnitsCount += card["count"]
      if card["ability"]:
        abilityCount += 1
      if card["attacks_ground"]:
        totalGroundT += 1
      if card["attacks_air"]:
        totalAirT += 1
      if card["target_only_buildings"]:
        totalBuildingT += 1
    # SPELL
    if card["type"] == "Spell":
      levelSum += int(card["level"], 10)
      if "duration" in card:
        avgSpellDuration += card["duration"]
      avgRange += card["radius"]
      if card["hits_ground"]:
        totalGroundT += 1
      if card["hits_air"]:
        totalAirT += 1
      if "appliesBuff" in card:
        if card["appliesBuff"]:
          totalBuffCount += 1
      if card["projectile"]:
        totalProjectiles += 1
      if "damage" in card:
        totalDMG += card["damage"]
      if "ignore_buildings" in card:
        if card["ignore_buildings"]:
          totalBuildingT += 1
    # BUILDING
    if card["type"] == "Building":
      levelSum += int(card["level"], 10)
      avgSpellDuration += card["duration"]
      avgRange += card["range"]
      totalHP += card["hitpoints"]
      if "damage" in card:
        totalDMG += card["damage"]
      if "hitspeed" in card:
        avgAS += card["hitspeed"]
      if card["attacks_ground"]:
        totalGroundT += 1
      if card["attacks_air"]:
        totalAirT += 1
    # END FOR
  # AVG CALCULATIONS
  if(match["troop"] > 0):
    avgSpeed = avgSpeed / match["troop"]
  if (match["troop"] + match["building"] > 0): 
    avgAS = avgAS / (match["troop"] + match["building"])
  if (match["spell"] + match["building"] > 0):
    avgSpellDuration = avgSpellDuration / (match["spell"] + match["building"])
  avgRange = avgRange / 8
  # END ELABORATIONS
  myDict["levelSum"] = levelSum
  myDict["avgSpeed"] = avgSpeed
  myDict["avgSpellDuration"] = avgSpellDuration
  myDict["totalHP"] = totalHP
  myDict["totalDMG"] = totalDMG
  myDict["avgAS"] = avgAS
  myDict["totalUnitsCount"] = totalUnitsCount
  myDict["abilityCount"] = abilityCount
  myDict["avgRange"] = avgRange
  myDict["totalGroundT"] = totalGroundT
  myDict["totalAirT"] = totalAirT
  myDict["totalBuildingT"] = totalBuildingT
  myDict["totalBuffCount"] = totalBuffCount
  myDict["totalProjectiles"] = totalProjectiles
  finalResult.append(myDict)

matches_df = pd.DataFrame(data=finalResult)
#version1: ranks.csv
matches_df.to_csv("../csv_sources/leveled_ranks.csv", header=True)
matches_df

,rank,troop,building,spell,common,rare,epic,legendary,elixir,conf_synergy,...,totalDMG,avgAS,totalUnitsCount,abilityCount,avgRange,totalGroundT,totalAirT,totalBuildingT,totalBuffCount,totalProjectiles
0,Challenger,5,0,3,1,1,4,2,3.375,2865.990531,...,1320,1.420000,19,2,2712.50,8,3,0,0,3
1,Challenger,5,1,2,1,2,3,2,4.250,3980.516610,...,1847,1.316667,5,3,2018.75,7,5,1,2,0
2,Master,7,1,0,3,3,2,0,4.875,0.000000,...,2100,2.075000,27,1,1925.00,8,2,2,0,0
3,Challenger,7,0,1,4,2,1,1,3.750,1389.332225,...,2068,1.428571,11,2,4175.00,8,6,0,0,1
4,Challenger,5,0,3,1,1,5,1,4.500,4049.013035,...,1688,1.740000,5,2,3056.25,8,4,1,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10495,Champion,5,1,2,0,4,4,0,3.625,2191.588585,...,1505,2.783333,5,4,2800.00,8,4,1,1,1
10496,Champion,6,0,2,1,2,3,2,4.000,26569.219252,...,2462,1.233333,6,4,2581.25,8,4,1,2,0
10497,Champion,6,0,2,2,1,2,3,3.750,2671.736517,...,2095,1.316667,10,3,2275.00,8,4,1,1,1
10498,Champion,5,1,2,4,2,1,1,3.125,14365.842341,...,2351,1.733333,17,0,3868.75,8,3,1,0,2
